# Minifloat and MX Example

Work in progress examples to show how to use minifloat and MX with Brevitas

In [3]:
from brevitas.quant.experimental.float_base import Fp8e4m3Mixin
from brevitas.quant.experimental.mx_quant_ocp import MXFloatWeight
from brevitas.quant.experimental.float_quant_ocp import FpOCPWeightPerTensorFloat, FpOCPActPerTensorFloat
from brevitas.quant.experimental.mx_quant_ocp import MXFloatAct
import brevitas.nn as qnn
import torch.nn as nn
import torch
from brevitas.quant_tensor import FloatQuantTensor

class OCPFP8Weight(FpOCPWeightPerTensorFloat, Fp8e4m3Mixin):
    pass


class OCPFP8Act(FpOCPActPerTensorFloat, Fp8e4m3Mixin):
    pass


class FP8Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = qnn.QuantConv2d(32, 64, 3, weight_quant=OCPFP8Weight, input_quant=OCPFP8Act)
    
    def forward(self, x):
        return self.conv(x)

ocp_fp8_model = FP8Model()
x = torch.randn(1, 32, 8, 8)
ocp_fp8_model.eval()
o = ocp_fp8_model(x)

intermediate_input = ocp_fp8_model.conv.input_quant(x)
assert isinstance(intermediate_input, FloatQuantTensor)

/home/giuseppe/miniconda3/envs/brevitas_dev/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at /opt/conda/conda-bld/pytorch_1712608853099/work/torch/csrc/utils/python_arg_parser.cpp:294.)
  return F.conv2d(input, weight, bias, self.stride,


In [5]:
from brevitas.quant_tensor import GroupwiseFloatQuantTensor


class MXFloat8Weight(MXFloatWeight, Fp8e4m3Mixin):
    # The group dimension for the weights it is automatically identified based on the layer type
    # If a new layer type is used, it can be manually specified
    pass

class MXFloat8Act(MXFloatAct, Fp8e4m3Mixin):
    # It is necessary to specify the group dimension for the activation quantization
    group_dim = 1


class MXModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = qnn.QuantConv2d(32, 64, 3, weight_quant=MXFloat8Weight, input_quant=MXFloat8Act)
    
    def forward(self, x):
        return self.conv(x)

mx_model = MXModel()
x = torch.randn(1, 32, 8, 8)
mx_model.eval()
o = mx_model(x)

intermediate_input = mx_model.conv.input_quant(x)
assert isinstance(intermediate_input, GroupwiseFloatQuantTensor)